In [1]:
#Load packages
import requests
from bs4 import BeautifulSoup
import os
import json
import pandas as pd
import inspect
import collections
from datetime import datetime as dt
import time
import re
import ast

In [2]:
response = requests.get(url='https://store.steampowered.com/appreviews/413150?json=1').json()

In [3]:
def get_reviews(appid, params={'json':1}):
        url = 'https://store.steampowered.com/appreviews/'
        response = requests.get(url=url+appid, params=params, headers={'User-Agent': 'Mozilla/5.0'})
        return response.json()


In [4]:
def get_n_reviews(appid, n=100):
    reviews = []
    cursor = '*'
    params = {
            'json' : 1,
            'filter' : 'all',
            'language' : 'english',
            'day_range' : 9223372036854775807,
            'review_type' : 'all',
            'purchase_type' : 'all'
            }

    while n > 0:
        params['cursor'] = cursor.encode()
        params['num_per_page'] = min(100, n)
        n -= 100

        response = get_reviews(appid, params)
        cursor = response['cursor']
        reviews += response['reviews']

        if len(response['reviews']) < 100: break

    return reviews

In [5]:
#def get_app_id(game_name):
#    response = requests.get(url=f'https://store.steampowered.com/search/?term={game_name}&category1=998', headers={'User-Agent': 'Mozilla/5.0'})
#    soup = BeautifulSoup(response.text, 'html.parser')
#    app_id = soup.find(class_='search_result_row')['data-ds-appid']
#    
#    return app_id

In [6]:
#To get a bunch of app IDs at once
def get_n_appids(n=100, filter_by='topsellers'):
    appids = []
    url = f'https://store.steampowered.com/search/?category1=998&filter={filter_by}&page='
    page = 0

    while page*25 < n:
        page += 1
        response = requests.get(url=url+str(page), headers={'User-Agent': 'Mozilla/5.0'})
        soup = BeautifulSoup(response.text, 'html.parser')
        for row in soup.find_all(class_='search_result_row'):
            appids.append(row['data-ds-appid'])

    return appids[:n]

In [7]:
#see how long it takes to execute this cell
start = time.time()

reviews = []
appids = get_n_appids(100) #100 app/"game" IDs, 100 top sellers?
for appid in appids:
    reviews += get_n_reviews(appid, 100)


end = time.time()

total_time = end - start
print("\n" + str(total_time))


54.99268412590027


In [8]:
#want to see how reviews looks like.
reviews

#first of all can look up who wrote the review/go to the actual review on Steam by looking up 'steamid'
#might be good to only look at reviews with substantial playtime_forever & playtime_last_two_weeks -> need to parse from "author"
#also 'votes_up' to see how many people found it helpful & votes_funny
#weighted_vote_score - might be good to pull it in to see how it varies

[{'recommendationid': '82020816',
  'author': {'steamid': '76561198110513339',
   'num_games_owned': 2696,
   'num_reviews': 7,
   'playtime_forever': 149262,
   'playtime_last_two_weeks': 39,
   'playtime_at_review': 146079,
   'last_played': 1675217310},
  'language': 'english',
  'review': "After 8 years playing it, I didn't improve my skills in-game.\n\nHowever, I learned new language skills: now I can curse in Russian and Brazilian Portuguese.\n\nAlso, my tolerance level to guttural screams and loud sounds was increased.\n\nNow I can bear the crying of my children for longer and I can make them feel better by singing sweet traditional songs from other cultures, such as Очи чёрные (Dark Eyes) or Garota de Ipanema (The Girl from Ipanema).\n\nIn conclusion, Counter Strike transformed me into a better father and a better person, in addition to making me more patient, tolerant to hateful people and more trained in languages and cultures.\n\nBetter than any psychiatric therapy. 10/10",


In [9]:
df_init = pd.DataFrame(reviews)[['review','author','votes_up','votes_funny','weighted_vote_score', 'voted_up']]


In [10]:
df_init.shape

(9467, 6)

In [11]:
df_init.head()

,review,author,votes_up,votes_funny,weighted_vote_score,voted_up
0,"After 8 years playing it, I didn't improve my ...","{'steamid': '76561198110513339', 'num_games_ow...",1614,1462,0.971687376499176025,True
1,I NOT CAN PLAY,"{'steamid': '76561198058211132', 'num_games_ow...",2300,196,0.971291542053222656,False
2,This community is so nice i got a lot of tips ...,"{'steamid': '76561198158022300', 'num_games_ow...",1342,819,0.968938231468200684,True
3,Your team in every random competitive game:\n\...,"{'steamid': '76561198388416030', 'num_games_ow...",1696,405,0.966444194316864014,True
4,The Best game on steam but it has the worst an...,"{'steamid': '76561198815361023', 'num_games_ow...",500,3,0.965640485286712646,True


In [12]:
#Need to parse out the data from the author field
pd.set_option('display.max_colwidth', None)
print(df_init['author'].head(1)) #example of what this author column looks like.

0    {'steamid': '76561198110513339', 'num_games_owned': 2696, 'num_reviews': 7, 'playtime_forever': 149262, 'playtime_last_two_weeks': 39, 'playtime_at_review': 146079, 'last_played': 1675217310}
Name: author, dtype: object


### Extracting more data from the author column

In [13]:
#Convert this column to list first
df_init['author'].head()

0    {'steamid': '76561198110513339', 'num_games_owned': 2696, 'num_reviews': 7, 'playtime_forever': 149262, 'playtime_last_two_weeks': 39, 'playtime_at_review': 146079, 'last_played': 1675217310}
1       {'steamid': '76561198058211132', 'num_games_owned': 384, 'num_reviews': 7, 'playtime_forever': 101658, 'playtime_last_two_weeks': 0, 'playtime_at_review': 87354, 'last_played': 1617129575}
2         {'steamid': '76561198158022300', 'num_games_owned': 255, 'num_reviews': 33, 'playtime_forever': 1254, 'playtime_last_two_weeks': 0, 'playtime_at_review': 1254, 'last_played': 1567873215}
3     {'steamid': '76561198388416030', 'num_games_owned': 1, 'num_reviews': 1, 'playtime_forever': 438635, 'playtime_last_two_weeks': 2430, 'playtime_at_review': 436409, 'last_played': 1675798503}
4        {'steamid': '76561198815361023', 'num_games_owned': 7, 'num_reviews': 6, 'playtime_forever': 204951, 'playtime_last_two_weeks': 0, 'playtime_at_review': 181968, 'last_played': 1641123812}
Name: author, d

In [14]:
author_fields = df_init['author'].values.tolist()

In [15]:
author_df = pd.DataFrame(author_fields, columns = ['steamid','num_games_owned','num_reviews','playtime_forever',
                                                   'playtime_last_two_weeks','playtime_at_review','last_played'])

In [16]:
author_df.shape

(9467, 7)

In [17]:
df = pd.concat([df_init, author_df], axis=1)

In [22]:
#df.head() #the author columns are included now

In [19]:
df.columns

Index(['review', 'author', 'votes_up', 'votes_funny', 'weighted_vote_score',
       'voted_up', 'steamid', 'num_games_owned', 'num_reviews',
       'playtime_forever', 'playtime_last_two_weeks', 'playtime_at_review',
       'last_played'],
      dtype='object')

In [20]:
#subset and rearrange columns
#won't really need "last_played"
df = df[['steamid','review','num_games_owned','num_reviews','playtime_forever','playtime_last_two_weeks','playtime_at_review',
         'votes_up','votes_funny','weighted_vote_score','voted_up']]

In [21]:
df.head()
#last played value is still weird

,steamid,review,num_games_owned,num_reviews,playtime_forever,playtime_last_two_weeks,playtime_at_review,votes_up,votes_funny,weighted_vote_score,voted_up
0,76561198110513339,"After 8 years playing it, I didn't improve my skills in-game.\n\nHowever, I learned new language skills: now I can curse in Russian and Brazilian Portuguese.\n\nAlso, my tolerance level to guttural screams and loud sounds was increased.\n\nNow I can bear the crying of my children for longer and I can make them feel better by singing sweet traditional songs from other cultures, such as Очи чёрные (Dark Eyes) or Garota de Ipanema (The Girl from Ipanema).\n\nIn conclusion, Counter Strike transformed me into a better father and a better person, in addition to making me more patient, tolerant to hateful people and more trained in languages and cultures.\n\nBetter than any psychiatric therapy. 10/10",2696,7,149262,39,146079,1614,1462,0.971687376499176025,True
1,76561198058211132,I NOT CAN PLAY,384,7,101658,0,87354,2300,196,0.971291542053222656,False
2,76561198158022300,This community is so nice i got a lot of tips on how to uninstall the game and kill myself,255,33,1254,0,1254,1342,819,0.968938231468200684,True
3,76561198388416030,Your team in every random competitive game:\n\n- You\n- Russian Guy\n- Another Russian Guy\n- A 6 year old kid who slept with your mother\n- Russian Guy who speaks English\n\n10/10,1,1,438635,2430,436409,1696,405,0.966444194316864014,True
4,76561198815361023,The Best game on steam but it has the worst anticheat.,7,6,204951,0,181968,500,3,0.965640485286712646,True


In [23]:
#Proportion of reviews with thumbs up or down
df['voted_up'].value_counts()

True     7633
False    1834
Name: voted_up, dtype: int64

In [24]:
#examples of reviews that received a thumbs down
df[df['voted_up']==False]

,steamid,review,num_games_owned,num_reviews,playtime_forever,playtime_last_two_weeks,playtime_at_review,votes_up,votes_funny,weighted_vote_score,voted_up
1,76561198058211132,I NOT CAN PLAY,384,7,101658,0,87354,2300,196,0.971291542053222656,False
9,76561198823299066,"[h1]ɢʀᴇᴀᴛ ɢᴀᴍᴇ, ʀᴜɪɴᴇᴅ ʙʏ ᴄʜᴇᴀᴛᴇʀꜱ[/h1]",135,1,2403817,18455,731994,1367,176,0.961813628673553467,False
63,76561198354594907,Too many hackers and toxic plebs.,1189,10,134415,348,3397,170,18,0.92324906587600708,False
66,76561198243124536,"ORIGINAL REVIEW Posted: Nov 27, 2016:\nExtremely fun and addictive game! That IS if you practice and learn the way the game is supposed to be played.\n\nActually requires a lot of patience, aim control, learning of map locations, etc.\n\nEdit Dec 7, 2018: Stupid crazy full with hackers, now that it's F2P i feel like it's gonna get taken over by even more hackers... RIP 2012 CSGO.\n\nEDIT 2021/09/03: Some of you are morons here... Clearly a huge ""fan"" base of CSGO. I've gathered around 9,000+ hours in total (in my other account as well) No. I wasn't a ""Noob"" or had no skill. I was GE for a very long time.\n\nThis review is from 2018. After all the new updates came into place. I LOVED the game, It was my addiction. After those updates, everything went down hill. MORE toxicity, MORE blatant hackers, and I for one wasn't a big fan of the newest updates. So I decided to leave, updated this review with my most recent experience.\nReviews are meant to talk about your experience, Not to kiss ass and make things look good when they aren't.",1324,53,192809,0,145081,277,13,0.922410786151885986,False
75,76561198799665922,[h1][b]I NOT CAN PLAY[/b][/h1],13,1,702322,132,702189,943,131,0.919177114963531494,False
...,...,...,...,...,...,...,...,...,...,...,...
9444,76561197964112971,"I saw the Overwhelmingly Positive reviews for the game initially and as someone who loves tactical RPGs I was pretty excited. I played through the entire thing on normal, and before I posted this I also watched part of a friends play through on hard.\n\nThe overall mechanics are pretty standard tactics fare, although its hard to say how much they really affect the overall game-play. From a personal perspective I felt the game was too easy in the beginning and the difficulty in hard mode came from being more irritating than tactical. \n\nThe real problem with the game is the story and writing. It feels like the plot was written by Asylum Films - a facade of a game story that's just aping other tactics games . They knew what story beats SHOULD be in the game, but not how to make them interesting or how to get from one to another naturally. Each time something would happen I kept hoping that this was going to be change or twist, but it takes the worst option at any turn. SPOILERS: [spoiler] The main character dies but then turns out to be fantasy Jesus, his childhood friend turns out to be the long lost king of a nation, You get betrayed by your friend but it is IMMEDIATELY resolved and has no relevance, etc. [/spoiler]\n\nThe romances are even worse. There's almost no lead up to any of them, and most of the characters have at max 1 conversation before falling in love, and then they rarely if ever say anything afterward. A lot of the dialogue gives off weird incel vibes too:\nSPOILERS: https://imgur.com/uPfhFIW\nSPOILERS: https://imgur.com/8fkEyKZ\nSPOILERS: https://imgur.com/rZHfYkz\n\nA lot of the writing also felt like someone's fetish material. SPOILERS: [spoiler] Diana, one of the main characters, starts out wearing plate mail (although it has boobs), then gets a costume change where she grows super huge and starts wearing a leotard. [/spoiler]\nSPOILERS: https://imgur.com/tZneDcc\n\nIn summary, while the game is fine the story and writing actually drag it down. A day or so after finishing it I'm basically left feeling disappointed.",293,3,1715,0,1715,27,1,0.661213696002960205,False
9448,76561197992861786,"This game's ""normal"" difficulty is a joke. I'm 4 hours 

#### Import NLP libraries

In [26]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords

#### Next: lowercase, remove special characters/only keep letters, remove stop words, etc.

In [27]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

#Create a function that tries to clean the reviews
def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower() #lowercase
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext) #remove weblinks
    rem_num = re.sub('[0-9]+', '', rem_url) #remove numbers
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')] #remove stop words
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)




In [28]:
df['clean_review']=df['review'].map(lambda s:preprocess(s)) 

In [31]:
df.shape

(9467, 12)

#### Remove any remaining non-English words

In [32]:
df = df[df['clean_review'].map(lambda x: x.isascii())]


In [33]:
df.shape #Removed some reviews that had foreign characters, not many

(9373, 12)

In [34]:
df.head()

,steamid,review,num_games_owned,num_reviews,playtime_forever,playtime_last_two_weeks,playtime_at_review,votes_up,votes_funny,weighted_vote_score,voted_up,clean_review
1,76561198058211132,I NOT CAN PLAY,384,7,101658,0,87354,2300,196,0.971291542053222656,False,play
2,76561198158022300,This community is so nice i got a lot of tips on how to uninstall the game and kill myself,255,33,1254,0,1254,1342,819,0.968938231468200684,True,community nice got lot tips uninstall game kill
3,76561198388416030,Your team in every random competitive game:\n\n- You\n- Russian Guy\n- Another Russian Guy\n- A 6 year old kid who slept with your mother\n- Russian Guy who speaks English\n\n10/10,1,1,438635,2430,436409,1696,405,0.966444194316864014,True,team every random competitive game russian guy another russian guy year old kid slept mother russian guy speaks english
4,76561198815361023,The Best game on steam but it has the worst anticheat.,7,6,204951,0,181968,500,3,0.965640485286712646,True,best game steam worst anticheat
5,76561198036073172,This community is so nice i got a lot of tips on how to uninstall the game and kill myself,321,5,9827,248,8458,989,660,0.96542966365814209,True,community nice got lot tips uninstall game kill


In [36]:
#df['steamid'].nunique()

In [38]:
#Again, reorder and subset columns
df = df[['clean_review','num_games_owned','num_reviews','playtime_forever','playtime_last_two_weeks','playtime_at_review',
    'votes_up','votes_funny','weighted_vote_score','voted_up']]

,clean_review,num_games_owned,num_reviews,playtime_forever,playtime_last_two_weeks,playtime_at_review,votes_up,votes_funny,weighted_vote_score,voted_up
1,play,384,7,101658,0,87354,2300,196,0.971291542053222656,False
2,community nice got lot tips uninstall game kill,255,33,1254,0,1254,1342,819,0.968938231468200684,True
3,team every random competitive game russian guy another russian guy year old kid slept mother russian guy speaks english,1,1,438635,2430,436409,1696,405,0.966444194316864014,True
4,best game steam worst anticheat,7,6,204951,0,181968,500,3,0.965640485286712646,True
5,community nice got lot tips uninstall game kill,321,5,9827,248,8458,989,660,0.96542966365814209,True
...,...,...,...,...,...,...,...,...,...,...
9462,finished game normal difficulty must say loved like bit era graphic style game played games like tactics ogre fire emblem bahamut lagoon game made remember lot shining force etc dear guess younger generations could little put dated graphics outdated always style interface could better endend playing game controller instead keyboard mouse plan use controller know little bit frustrating many extra clicks anything certain information behind clicks gameplay main thing game addictive srpite map represents squad squad size limited leader leadership example leadership capacity soldier costs capacity higher tier capacity takes soldier gain experience class points experience raises stats class points allow change class reach certain ammount change class many times want long gather enough points change class linear might want upgrade fighter archer rogue squirmicher mounted scout apprentice etc choose scout get upgraded hussar ranger branching choice squads equiped artifact unlock possibility equip artifacts artifacts weapons armor etc wich raise certain stats give abilities example damage soldiers first row also items earn buy use individual soldiers giving change soldiers affinity earth fire light etc even traits example executioner enemy soldier chance get extra turn really lot tinker army choice different soldiers disposition grid class upgrades items artifacts relationships friend bonded status might give extra abilities much changed actually change squad fights role battlefield matter damage actually change tactics always anxious try see results battlefield find really cool synergies specially able bigger squads let fact get tier soldiers reach middle game make thing evolution progress stagnate game really chapter map chapters story short cut scenes never felt dull wanting skip menu organize army access tech tree also opens new abilities features marketplace recruit buy sell items artifacts resources also relationship screen couple conversations characters might appear lead friedships even romantic bonds nothing special romantic part seriously underdevloped almost like token end get words main characters reached character bonded even mention anything nothing happened story gets things going done nothing special nothing really boring unlike mercenaries series tactical games end skipping scenes battles enjoyed game much hours time review played last days one days went bed hooked let finally finished drag also feel short certainyl looking forward sequel question feel free ask,363,18,1438,0,1438,9,0,0.630285263061523438,True
9463,picked friend mine said fantastic played demo figured picked streamed whim two days day hour game session find hard put game love letter love tactics advance wars fire emblem least good parts mixed mashed together make arguably one best games played rate one hero hour might magic formula took improved games know genre took focused parts people loved brought back modern audience low price tag complaints far chapter menu navigation handling unit formations squads could use work works well enough slow much dialogue spelling errors caught mostly end complaints rest good good play,189,4,917,0,737,12,0,0.630183041095733643,True
9464,executive summary fire emblem battle wesnoth worse interfac